In [1]:
import os
from pprint import pprint
from pathlib import Path
from copy import deepcopy

import numpy as np
from scipy.constants import pi
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from matplotlib import rcParams
rcParams["font.size"] = 18

from quvac.postprocess import integrate_spherical
from quvac.plotting import plot_mollweide
from quvac.utils import write_yaml, read_yaml

SCRIPT_PATH = '../src/quvac/simulation.py'

path = "../data/tutorials/tutorial_4"
Path(path).mkdir(parents=True, exist_ok=True)

## Parallel simulations

quvac can run a simulation in parallel by splitting the discretized time integral between different jobs and then collecting results together. A separate script and command are responsible for such computation. To control the parameters of parallelization (number of jobs, parameters of each job), a separate section in `ini.yml` is required.

Usually such parallelization makes sense only on cluster enviroments.

We would calculate the same counter-propagating setup.

The parallel calculation is launched on the local machine (this is controlled by `cluster`: 'local' or 'slurm'), so the speedup depends on the machine's characteristics. To launch it on a cluster, change 'local' -> 'slurm' and provide necessary sbatch parameters (they are similar to stardard sbatch scripts). Look up keyword arguments for [submitit](https://github.com/facebookincubator/submitit).

In [2]:
lam = 800e-9
w0 = 2 * lam
tau = 25e-15

# all parameters in SI units
field_1_params = {
    "field_type": "paraxial_gaussian_maxwell",
    "focus_x": [0.,0.,0.],
    "focus_t": 0.,
    "theta": 0,
    "phi": 0,
    "beta": 0,
    "phase0": 0,
    "lam": lam,
    "w0": w0,
    "tau": tau,
    "W": 25,
    "order": 0,
}

# add counter-propagating field
# (it has the same parameters apart from propagation direction and polarization)
field_2_params = deepcopy(field_1_params)
field_2_params["theta"] = 180
field_2_params["beta"] = 45

# combine
fields_params = {
    "field_1": field_1_params,
    "field_2": field_2_params,
}

# grid parameters
grid_params = {
    'mode': 'dynamic',
    'collision_geometry': 'z',
    'transverse_factor': 20,
    'longitudinal_factor': 8,
    'time_factor': 4,
    'spatial_resolution': 1,
    'time_resolution': 1,
}

# cluster parameters <---- NEW
cluster_params = {
    'n_jobs': 2,
    'cluster': 'local',
    'sbatch_params': {
        'cpus_per_task': 2,
        # 'slurm_mem': '50GB',
        'timeout_min': 60,
    },
}

ini_data = {
    'mode': 'simulation',
    'fields': fields_params,
    'grid': grid_params,
    'integrator': {
        'type': 'vacuum_emission',
    },
    'performance': {
        'nthreads': 2,
    },
    'cluster_params': cluster_params,
}

folder_seq = "sequential"
folder_par = "parallel"    

In [3]:
def launch_simulation(folder,n_jobs):
    ini_data['cluster_params']['n_jobs'] = n_jobs
    
    path_local = os.path.join(path, folder)
    Path(path_local).mkdir(parents=True, exist_ok=True)
    ini_file = os.path.join(path_local, 'ini.yml')
    write_yaml(ini_file, ini_data)

    status = os.system(f"quvac-simulation-parallel --input {ini_file}")

In [4]:
launch_simulation(folder_seq, n_jobs=1)


Timings:
Run jobs:                     1 min 9.13 s
Postprocess:                        0.00 s
----------------------------------------------------
Total:                        1 min 9.13 s

Memory (max usage):
Running jobs:                    139.08 MB
Total:                           139.08 MB

Simulation finished!


In [8]:
launch_simulation(folder_par, n_jobs=2)


Timings:
Run jobs:                    1 min 56.36 s
Postprocess:                        0.00 s
----------------------------------------------------
Total:                       1 min 56.36 s

Memory (max usage):
Running jobs:                    138.93 MB
Total:                           138.93 MB

Simulation finished!
